<a href="https://colab.research.google.com/github/Heba2424/DEPI_Projects/blob/main/LLMs_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install simpletransformers --quiet
! pip install sentence-transformers --quiet
! pip show pyarrow --quiet
! pip install pyarrow==6.0.1 --quiet
!pip install datasets --quiet
!pip install transformers datasets torch sklearn --quiet


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.15.0 requires pyarrow>=8.0.0, but you have pyarrow 6.0.1 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 6.0.1 which is incompatible.
datasets 2.21.0 requires pyarrow>=15.0.0, but you have pyarrow 6.0.1 which is incompatible.
streamlit 1.37.1 requires pyarrow>=7.0, but you have pyarrow 6.0.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info di

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel , ClassificationArgs
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sentence_transformers import SentenceTransformer , util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch

In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
!git clone https://huggingface.co/datasets/sepidmnorozy/Arabic_sentiment

Cloning into 'Arabic_sentiment'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16 (from 1)
Unpacking objects: 100% (16/16), 150.22 KiB | 5.18 MiB/s, done.


In [ ]:
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/sepidmnorozy/Arabic_sentiment

fatal: destination path 'Arabic_sentiment' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import torch


# Load datasets
splits = {'train': 'train.csv', 'validation': 'dev.csv', 'test': 'test.csv'}
train_df = pd.read_csv("/content/Arabic_sentiment/" + splits["train"])
val_df = pd.read_csv("/content/Arabic_sentiment/" + splits["validation"])
test_df = pd.read_csv("/content/Arabic_sentiment/" + splits["test"])

# Display the first few rows
print(train_df.head())
print(val_df.head())
print(test_df.head())


   label                                               text
0      1                   ربك دايما جنبك لو نديته هتلاقيه 
1      0                                               بعئد
2      0  إذا استمرت القيادة المصرية الحالية في نهجها فس...
3      0                                     #توقف_الواتساب
4      1  المعنى الحقيقي للوطنية المعنى الحقيقي للضحكة ا...
   label                                               text
0      1  ياريت حدا عم يواسي السوريين بغربتهن للاسف باب ...
1      1  لمن يعتقد ان صداقتي بجون ستيوارت اليهودي تهمة,...
2      1  أشارك اليوم في مبادرة فودافون لمحو الأمية, الع...
3      0   يضحكون و م شاففوا انفسهم ، لو قلنا غنوا جابوا...
4      0                          حرام عليك كانو بقرة بتولد
   label                                               text
0      1  سلامة أحمد سلامة، تركت إرثاً من الأخلاق حين عز...
1      1  قرار رئيس الجمهورية بعودة البرلمان للانعقاد صح...
2      1                             صباح الخيرات والعبادات
3      1                                

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for param in model.bert.parameters():
    param.requires_grad = False


In [ ]:
from datasets import Dataset
from transformers import BertTokenizer

# Convert pandas DataFrames to Hugging Face Datasets
def create_hf_dataset(df):
    return Dataset.from_dict({
        'text': df['text'].tolist(),
        'label': df['label'].tolist()
    })

# Create datasets
train_dataset = create_hf_dataset(train_df)
val_dataset = create_hf_dataset(val_df)
test_dataset = create_hf_dataset(test_df)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch or TensorFlow
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])


Map:   0%|          | 0/2468 [00:00<?, ? examples/s]

Map:   0%|          | 0/353 [00:00<?, ? examples/s]

Map:   0%|          | 0/706 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments

# Define TrainingArguments with learning rate scheduler
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,  # Adjust learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,  # Increase number of epochs
    weight_decay=0.01,
    logging_dir='./logs',
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments ,default_data_collator

# Use default_data_collator
data_collator = default_data_collator

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)


# Start training
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.655984
2,No log,0.500820
3,No log,0.463445
4,0.570300,0.514233


TrainOutput(global_step=620, training_loss=0.5263612870247133, metrics={'train_runtime': 966.6349, 'train_samples_per_second': 10.213, 'train_steps_per_second': 0.641, 'total_flos': 2597432338513920.0, 'train_loss': 0.5263612870247133, 'epoch': 4.0})

In [ ]:
# Evaluate model
results = trainer.evaluate(test_dataset)

# Print evaluation results
print("Results on test dataset:", results)

Results on test dataset: {'eval_runtime': 23.6431, 'eval_samples_per_second': 29.861, 'eval_steps_per_second': 1.903, 'epoch': 4.0}


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import torch

# Make predictions on the validation set
def get_predictions(trainer, dataset):
    predictions = trainer.predict(dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    return preds, predictions.label_ids

# Get predictions and true labels
val_preds, val_labels = get_predictions(trainer, val_dataset)

# Generate confusion matrix
conf_matrix = confusion_matrix(val_labels, val_preds)
print("Confusion Matrix:")
print(conf_matrix)

# Generate classification report
report = classification_report(val_labels, val_preds, target_names=['class_0', 'class_1'])
print("Classification Report:")
print(report)


Confusion Matrix:
[[141  41]
 [ 31 140]]
Classification Report:
              precision    recall  f1-score   support

     class_0       0.82      0.77      0.80       182
     class_1       0.77      0.82      0.80       171

    accuracy                           0.80       353
   macro avg       0.80      0.80      0.80       353
weighted avg       0.80      0.80      0.80       353



In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Set model to evaluation mode
model.eval()

def get_bert_embeddings(texts, tokenizer, model, max_length=128):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)
    # Take the mean of the token embeddings (or you could use the CLS token embedding)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Example usage
texts = ["I love machine learning", "BERT is amazing"]
embeddings = get_bert_embeddings(texts, tokenizer, model)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load your data
splits = {'train': 'train.csv', 'validation': 'dev.csv', 'test': 'test.csv'}
train_df = pd.read_csv("/content/Arabic_sentiment/" + splits["train"])
val_df = pd.read_csv("/content/Arabic_sentiment/" + splits["validation"])
test_df = pd.read_csv("/content/Arabic_sentiment/" + splits["test"])

# Get embeddings for train, validation, and test sets
X_train = get_bert_embeddings(train_df['text'].tolist(), tokenizer, model)
X_val = get_bert_embeddings(val_df['text'].tolist(), tokenizer, model)
X_test = get_bert_embeddings(test_df['text'].tolist(), tokenizer, model)

y_train = train_df['label']
y_val = val_df['label']
y_test = test_df['label']

# Split train and validation sets if needed
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Logistic Regression
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))

# Optionally evaluate on test set
y_test_pred = clf.predict(X_test)
print(classification_report(y_test, y_test_pred))


              precision    recall  f1-score   support

           0       0.72      0.75      0.74       122
           1       0.75      0.71      0.73       125

    accuracy                           0.73       247
   macro avg       0.73      0.73      0.73       247
weighted avg       0.73      0.73      0.73       247

              precision    recall  f1-score   support

           0       0.73      0.75      0.74       355
           1       0.74      0.72      0.73       351

    accuracy                           0.73       706
   macro avg       0.73      0.73      0.73       706
weighted avg       0.73      0.73      0.73       706



In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity between train and test embeddings
similarities = cosine_similarity(X_train, X_test)

# Get the average similarity for evaluation
average_similarity = np.mean(similarities)
print(f"Average Cosine Similarity: {average_similarity}")


Average Cosine Similarity: 0.9153980016708374


In [ ]:
from sklearn.metrics import pairwise_distances

def evaluate_similarity(embeddings, labels):
    unique_labels = np.unique(labels)
    class_similarities = {}
    for label in unique_labels:
        class_indices = np.where(labels == label)[0]
        class_embeddings = embeddings[class_indices]
        # Compute pairwise distances within class
        distances = pairwise_distances(class_embeddings, class_embeddings, metric='cosine')
        avg_distance = np.mean(distances)
        class_similarities[label] = avg_distance
    return class_similarities

train_labels = train_df['label'].tolist()
test_labels = test_df['label'].tolist()

train_class_similarities = evaluate_similarity(X_train, y_train)
test_class_similarities = evaluate_similarity(X_test, y_test)

print("Train Class Similarities:", train_class_similarities)
print("Test Class Similarities:", test_class_similarities)


Train Class Similarities: {0: 0.09163141, 1: 0.07688348}
Test Class Similarities: {0: 0.08345579, 1: 0.08286472}
